In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pyautogui
import math

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                      max_num_hands=1,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Screen dimensions
screen_w, screen_h = pyautogui.size()

# Camera setup
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        continue
    
    # Flip and convert color
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    
    # Draw hand landmarks
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Get landmark coordinates
            landmarks = hand_landmarks.landmark
            
            # Get index and thumb tip coordinates
            index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP]
            
            # Convert to screen coordinates
            x = int(index_tip.x * screen_w)
            y = int(index_tip.y * screen_h)
            
            # Move mouse
            pyautogui.moveTo(x, y)
            
            # Calculate distance between index and thumb
            distance = math.sqrt((index_tip.x - thumb_tip.x)**2 + 
                               (index_tip.y - thumb_tip.y)**2)
            
            # Click if distance is small
            if distance < 0.05:
                pyautogui.click()
    
    cv2.imshow('Hand Gesture Mouse', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()